In [ ]:
# !pip install langchain
# !pip install langchain-community
# !pip install langchain-openai

In [ ]:
# uncomment this if running locally
# from dotenv import load_dotenv

# load_dotenv()

# FILEPATH: /Users/greatmaster/Desktop/projects/oreilly-live-trainings/oreilly_live_training_agents/notebooks/1.3-intro-agents-with-langchain.ipynb
# SET YOUR API KEYS FOR LANGSMITH AND TAVILY TO RUN THIS NOTEBOOK!
# langsmith: https://docs.smith.langchain.com/#:~:text=install%20-U%20langsmith-,Create%20an%20API%20key,click%20Create%20API%20Key.,-Setup%20your%20environment
# tavily: https://app.tavily.com/sign-in
# openai: https://platform.openai.com/docs/quickstart
# import os
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = "your-api-key"
# os.environ["TAVILY_API_KEY"] = "your-api-key"
# os.environ["OPENAI_API_KEY"] = 'your-api-key'

Personal Assistant Agent
- Accees to emails
- acceess to my terminal for controlling different things like creating files etc...
- Have the assistant create our tasks for us

In [1]:
from langchain.agents import create_tool_calling_agent
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.tools import tool
from langchain.agents import AgentExecutor
from langchain_community.document_loaders import PyPDFLoader
import subprocess
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import subprocess

In [2]:
@tool
def send_email(destination_email, message_content, subject_content):
    "This function takes in destination email and the contents and sends out an email."
    # Sender email address
    sender_email = "lucasenkrateia@gmail.com"

    # Email Subject
    subject = subject_content

    # Email body
    body = message_content

    # SMTP server credentials
    smtp_server = "smtp.gmail.com"
    smtp_port = 587  # For starttls
    smtp_user = sender_email
    smtp_password = os.environ["GMAIL_PWD"]

    # Create a multipart message and set headers
    message = MIMEMultipart()
    message["From"] = sender_email
    message["To"] = destination_email
    message["Subject"] = subject

    # Add body to email
    message.attach(MIMEText(body, "plain"))

    # Convert message to string for sending
    text = message.as_string()

    # Send email
    try:
        # Connect to the server
        server = smtplib.SMTP(smtp_server, smtp_port)
        server.starttls()  # Secure the connection
        server.login(smtp_user, smtp_password)
        
        # Send the email
        server.sendmail(sender_email, destination_email, text)
        print("Email sent successfully")
    except Exception as e:
        # Print any error messages to stdout
        print(f"Error: {e}")
    finally:
        server.quit()


@tool
def execute_bash_command(command):
    """Executes a bash command."""
    result = subprocess.run(command, shell=True)
    output = result.stdout
    return output

@tool
def read_txt_tool(file_path):
    """Reads contents of a .txt file."""
    with open(file_path, "r") as f:
        contents = f.read()
    
    return contents

@tool
def summarize_pdf(pdfpath):
    """Summarizes the contents of a pdf"""
    loader = PyPDFLoader(pdfpath)
    documents = loader.load()
    contents = ""
    for doc in documents:
        contents += doc.page_content + "\n"
    
    summary = ChatOpenAI(temperature=0, model="gpt-4o").invoke(f"Summarize this content into simplified bullet points: {contents}")
    
    return summary.content
    
tools = [execute_bash_command, send_email, read_txt_tool, summarize_pdf]

In [3]:
prompt = hub.pull("hwchase17/openai-tools-agent")

prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-tools-agent', 'lc_hub_commit_hash': 'c18672812789a3b9697656dd539edf0120285dcae36396d0b548ae42a4ed66f5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlacehold

In [4]:
llm = ChatOpenAI(model="gpt-4o")

In [5]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [6]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
agent_executor.invoke({"input": "1. Create a file named tasks-for-lucas.txt containing 10 automation tasks you find interesting. 2. Send email to '''lucasbnsoares@hotmail.com''' with the top 3 tasks from that file.",
                       "agent_scratchpad": []})



> Entering new AgentExecutor chain...

Invoking: `execute_bash_command` with `{'command': "echo '1. Automate data entry in spreadsheets\n2. Set up automated email responses\n3. Automate backups of important files\n4. Schedule social media posts\n5. Automate invoice generation\n6. Monitor website uptime and send alerts\n7. Automate data scraping from websites\n8. Schedule system cleanups\n9. Automate sending out customer surveys\n10. Automate report generation from databases' > tasks-for-lucas.txt"}`


None
Invoking: `read_txt_tool` with `{'file_path': 'tasks-for-lucas.txt'}`


1. Automate data entry in spreadsheets
2. Set up automated email responses
3. Automate backups of important files
4. Schedule social media posts
5. Automate invoice generation
6. Monitor website uptime and send alerts
7. Automate data scraping from websites
8. Schedule system cleanups
9. Automate sending out customer surveys
10. Automate report generation from databases

Invoking: `send_email` with `{'destinati

{'input': "1. Create a file named tasks-for-lucas.txt containing 10 automation tasks you find interesting. 2. Send email to '''lucasbnsoares@hotmail.com''' with the top 3 tasks from that file.",
 'agent_scratchpad': [],
 'output': 'The file `tasks-for-lucas.txt` has been created with 10 interesting automation tasks, and an email with the top 3 tasks has been sent to lucasbnsoares@hotmail.com.'}

In [8]:
!cat tasks-for-lucas.txt

1. Automate the summarization of PDF documents for quick review.
2. Automate the process of converting speech to text for meeting notes.
3. Create a bot that automatically responds to specific emails based on predefined criteria.
4. Develop a system that automatically backs up important files to the cloud at scheduled intervals.
5. Implement a tool that tracks and reports on changes to specific websites or web pages.
6. Create a script that automatically organizes files in a folder based on their type or content.
7. Develop a program that can automatically edit videos based on user-defined parameters.
8. Automate the process of data entry from scanned documents into a database.
9. Create a bot that can automatically post content to social media platforms based on a schedule.
10. Implement an AI-driven assistant that can help manage and prioritize emails.


![agent-email-automations-top-3](assets-resources/agent-email-automations-top-3.png)

In [9]:
agent_executor.invoke({"input": "Create a folder named `lucas-test`. Inside that folder create a file named lucas-test.txt containing a random quote from a famous philosopher. Send an email to '''lucasbnsoares@hotmail.com''' with that quote.",
                             "agent_scratchpad": []})



> Entering new AgentExecutor chain...

Invoking: `execute_bash_command` with `{'command': "mkdir -p lucas-test && echo 'The unexamined life is not worth living. - Socrates' > lucas-test/lucas-test.txt && cat lucas-test/lucas-test.txt"}`


The unexamined life is not worth living. - Socrates
None
Invoking: `read_txt_tool` with `{'file_path': 'lucas-test/lucas-test.txt'}`


The unexamined life is not worth living. - Socrates

Invoking: `send_email` with `{'destination_email': 'lucasbnsoares@hotmail.com', 'message_content': 'The unexamined life is not worth living. - Socrates', 'subject_content': 'Philosophical Quote'}`


Email sent successfully
NoneThe folder `lucas-test` has been created with a file named `lucas-test.txt` containing the quote "The unexamined life is not worth living. - Socrates". An email with this quote has also been sent to `lucasbnsoares@hotmail.com`.

> Finished chain.


{'input': "Create a folder named `lucas-test`. Inside that folder create a file named lucas-test.txt containing a random quote from a famous philosopher. Send an email to '''lucasbnsoares@hotmail.com''' with that quote.",
 'agent_scratchpad': [],
 'output': 'The folder `lucas-test` has been created with a file named `lucas-test.txt` containing the quote "The unexamined life is not worth living. - Socrates". An email with this quote has also been sent to `lucasbnsoares@hotmail.com`.'}

![quote-socrates](assets-resources/quote-proof.png)

In [11]:
agent_executor.invoke({"input": """1. Read the contents of this pdf: located in ./assets-resources/tree-of-thoughts.pdf 2.Send a summary to '''lucasbnsoares@hotmail.com'''""",
                                "agent_scratchpad": []})



> Entering new AgentExecutor chain...

Invoking: `summarize_pdf` with `{'pdfpath': './assets-resources/tree-of-thoughts.pdf'}`


**Tree of Thoughts: Deliberate Problem Solving with Large Language Models**

- **Problem**: Current language models (LMs) like GPT-4 are limited by token-level, left-to-right decision-making, which hinders their performance in tasks requiring strategic planning or exploration.
- **Solution**: Introduce "Tree of Thoughts" (ToT) framework to enhance LMs by enabling exploration over coherent units of text ("thoughts") and allowing deliberate decision-making.
- **Key Features of ToT**:
  - **Thought Decomposition**: Break down the problem-solving process into intermediate steps or "thoughts".
  - **Thought Generation**: Generate multiple potential thoughts for each step.
  - **State Evaluation**: Evaluate the progress of each state towards solving the problem.
  - **Search Algorithms**: Use search algorithms like breadth-first search (BFS) or depth-first search

{'input': "1. Read the contents of this pdf: located in ./assets-resources/tree-of-thoughts.pdf 2.Send a summary to '''lucasbnsoares@hotmail.com'''",
 'agent_scratchpad': [],
 'output': 'The contents of the PDF have been summarized and sent to lucasbnsoares@hotmail.com. If you need any further assistance, feel free to ask!'}

![summary-email](./assets-resources/summary-email.png)